# 📘 cuda_optics: Complete Documentation

Welcome to the official reference for **cuda_optics**. This notebook serves as both a manual and an interactive playground.

## 📑 Index
1.  [Core Functions (Simulation & Plotting)](#1.-Core-Functions)
2.  [Beam Generation](#2.-Beam-Generation)
3.  [Screens & Apertures](#3.-Screens-&-Apertures)
4.  [Thin Lenses & Gratings](#4.-Thin-Lenses-&-Gratings)
5.  [Mirrors (Flat, Spherical, Parabolic)](#5.-Mirrors)
6.  [Refractive Elements (Slab, Thick Lens, Prism)](#6.-Refractive-Elements)
7.  [Analysis Tools](#7.-Analysis)
8.  [Complex Examples](#8.-Complex-Examples)

---

In [ ]:
import cuda_optics as co
import matplotlib.pyplot as plt
import numpy as np

print("cuda_optics loaded! Ready to simulate.")

## 1. Core Functions
These are the two most important functions you will use in every simulation.

### `co.run_simulation(rays, elements, bounces, use_gpu)`
The engine that calculates ray physics.
* **`rays`** *(list)*: List of `Ray` objects to simulate.
* **`elements`** *(list)*: List of `OpticalElement` objects in the scene.
* **`bounces`** *(int)*: Max number of interactions allowed per ray. (e.g., 5-10 for simple systems, 50+ for traps).
* **`use_gpu`** *(bool)*: If `True`, attempts to use CUDA. Falls back to C++ CPU if unavailable.

### `co.plot_system(rays, elements, ax)`
Visualizes the results on a Matplotlib axis.
* **`rays`** *(list)*: The rays (after simulation).
* **`elements`** *(list)*: The elements to draw.
* **`ax`** *(matplotlib.axes)*: The subplot to draw on.

## 2. Beam Generation
Instead of creating individual rays manually, use these helpers.

### A. `generate_parallel_beam`
Creates a collimated beam (infinite conjugate source).
* **`wavelengths`**: List of tuples `(nm, 'color')`.
* **`source_pos`**: `[x, y]` start point.
* **`target_element_centre`**: `[x, y]` point to aim at.
* **`target_element_aperture`**: Width of the beam.
* **`num_rays`**: Count.

### B. `generate_point_source`
Creates rays expanding from a single point (finite conjugate source).
* *(Same arguments as above)*

### C. `generate_gaussian_beam_deter`
Creates a parallel beam with Gaussian intensity profile (dense center).
* **`waist_radius`**: The beam width ($1/e^2$).
* *(Other arguments same as above)*

In [ ]:
# Example: Generating all 3 types
screen = co.Screen("Target", center=[1.0, 0.0], normal=[-1, 0], aperture=2.0)

# 1. Parallel (Green)
rays_para = co.generate_parallel_beam(
    wavelengths=[(550, 'green')],
    source_pos=[0.0, 0.5],
    target_element_centre=[1.0, 0.5],
    target_element_aperture=0.2,
    num_rays=10
)

# 2. Point Source (Red)
rays_point = co.generate_point_source(
    wavelengths=[(650, 'red')],
    source_pos=[0.0, 0.0],
    target_element_centre=[1.0, 0.0],
    target_element_aperture=0.4,
    num_rays=10
)

# 3. Gaussian (Blue)
rays_gauss = co.generate_gaussian_beam_deter(
    wavelengths=[(450, 'blue')],
    source_pos=[0.0, -0.5],
    target_element_centre=[1.0, -0.5],
    waist_radius=0.1,
    num_rays=15
)

all_rays = rays_para + rays_point + rays_gauss
co.run_simulation(all_rays, [screen], bounces=1, use_gpu=False)

fig, ax = plt.subplots(figsize=(8, 6))
co.plot_system(all_rays, [screen], ax)
plt.title("Beam Generators")
plt.show()

## 3. Screens & Apertures

### `Screen`
A detector plane. Rays that hit it are stopped and recorded.
* **`name`**: String ID.
* **`center`**: `[x, y]`.
* **`normal`**: `[nx, ny]` (Direction it faces).
* **`aperture`**: Width.

### `CircularAperture`
A stop/hole. Rays inside the radius pass; rays outside are blocked.
* **`radius`**: Size of the hole.
* **`outer_radius`**: Size of the blocking plate (default: 3x hole).

**Special Note:** Screens are the only element that 'captures' rays for `screen_result` analysis.

In [ ]:
stop = co.CircularAperture(
    name="Pinhole", 
    center=[0.5, 0.0], 
    normal=[1, 0], 
    radius=0.1, 
    outer_radius=0.4
)
screen = co.Screen("Detector", center=[1.0, 0.0], normal=[-1, 0], aperture=1.0)

rays = co.generate_parallel_beam([(550, 'green')], [0.0, 0.0], [0.5, 0.0], 0.6, 20)
co.run_simulation(rays, [stop, screen], bounces=2, use_gpu=False)

fig, ax = plt.subplots(figsize=(8, 4))
co.plot_system(rays, [stop, screen], ax)
plt.title("Aperture Blocking Rays")
plt.show()

## 4. Thin Lenses & Gratings
Idealized elements that are geometrically flat but alter ray angles.

### `ThinLens`
Focuses light without thickness/aberration calculations.
* **`focal_length`**: Positive for convex, negative for concave.

### `TransmissionGrating` & `ReflectiveGrating`
Diffracts light based on wavelength.
* **`lines_per_mm`**: Groove density (e.g., 600).
* **`order`**: Diffraction order (usually 1 or -1).

**Special Note:** `TransmissionGrating` passes light through. `ReflectiveGrating` bounces it back.

In [ ]:
grating = co.TransmissionGrating(
    name="Grating", center=[0.5, 0.0], normal=[1, 0], aperture=0.5, 
    lines_per_mm=600, order=1
)
lens = co.ThinLens(
    name="Focus Lens", center=[0.8, 0.2], normal=[1, -0.5], aperture=0.5, 
    focal_length=0.2
)

rays = co.generate_parallel_beam([(400, 'blue'), (700, 'red')], [0.0, 0.0], [0.5, 0.0], 0.1, 6)
co.run_simulation(rays, [grating, lens], bounces=5, use_gpu=False)

fig, ax = plt.subplots(figsize=(8, 5))
co.plot_system(rays, [grating, lens], ax)
plt.title("Grating Dispersion + Thin Lens")
plt.show()

## 5. Mirrors

### `FlatMirror`
Standard planar reflection.

### `SphericalMirror`
Curved surface defined by a radius.
* **`radius`**: Distance to center of curvature.
* **Special Note:** Suffers from Spherical Aberration (rays don't meet perfectly).

### `ParabolicMirror`
Ideal focusing shape.
* **`focal_length`**: Distance to focus.
* **Special Note:** Focuses parallel light to a perfect point.

In [ ]:
para = co.ParabolicMirror("Parabolic", [0.8, 0.2], [-1, 0], 0.4, focal_length=0.2)
sphere = co.SphericalMirror("Spherical", [0.8, -0.2], [-1, 0], 0.4, radius=0.4)

rays_1 = co.generate_parallel_beam([(550, 'cyan')], [0, 0.2], [0.8, 0.2], 0.35, 10)
rays_2 = co.generate_parallel_beam([(600, 'orange')], [0, -0.2], [0.8, -0.2], 0.35, 10)

co.run_simulation(rays_1 + rays_2, [para, sphere], bounces=2, use_gpu=False)

fig, ax = plt.subplots(figsize=(10, 6))
co.plot_system(rays_1 + rays_2, [para, sphere], ax)
plt.title("Parabolic (Top) vs Spherical (Bottom)")
plt.show()

## 6. Refractive Elements
These elements use Snell's Law and the internal glass database.
**Refractive Index Options:** `'BK7'`, `'F2'`, `'SF10'`, `'SiO2'`, or a float (e.g., `1.5`).

### `Slab`
Rectangular block of glass.
* **`thickness`**: Depth of the block.

### `Lens` (Thick)
Real lens with two curved surfaces.
* **`R1`**: Radius of front surface (+ means convex).
* **`R2`**: Radius of back surface (+ means concave).
* **`thickness`**: Center thickness.

### `Prism`
Equilateral prism for dispersion.
* **`apex_angle`**: Top angle (usually 60).
* **`side_length`**: Size of faces.

In [ ]:
prism = co.Prism(
    name="Prism", center=[0.5, 0.0], normal=[1, 0], 
    refractive_index='SF10', apex_angle=60, side_length=0.4
)
lens = co.Lens(
    name="Thick Lens", center=[0.0, 0.0], normal=[1, 0], aperture=0.5, 
    refractive_index=1.5, R1=0.5, R2=-0.5, thickness=0.2
)

rays = co.generate_point_source([(400, 'blue'), (700, 'red')], [-0.5, 0.1], [0.5, 0.0], 0.1, 5)
co.run_simulation(rays, [lens, prism], bounces=5, use_gpu=False)

fig, ax = plt.subplots(figsize=(10, 5))
co.plot_system(rays, [lens, prism], ax)
plt.title("Thick Lens focusing into a Prism")
plt.show()

## 7. Analysis

### `co.screen_result(screen, rays)`
Returns statistics about rays hitting a specific screen.
* **Returns**: Dictionary containing:
    * `rms_radius`: Root Mean Square radius of the spot.
    * `mean_pos`: [x, y] centroid.
    * `count`: Number of rays detected.

In [ ]:
screen = co.Screen("Sensor", center=[1.0, 0.0], normal=[-1, 0], aperture=1.0)
lens = co.ThinLens("Lens", center=[0.5, 0.0], normal=[1, 0], aperture=0.5, focal_length=0.5)

# Imperfect focus test
rays = co.generate_parallel_beam([(550, 'green')], [0,0], [0.5,0], 0.4, 20)
co.run_simulation(rays, [lens, screen], bounces=5, use_gpu=False)

# Get Stats
stats = co.screen_result(screen, rays)
print(f"RMS Radius: {stats['rms_radius']:.5f}")

## 8. Complex Examples

### Example A: Beam Expander (Keplerian)
Two positive lenses separated by the sum of their focal lengths ($f_1 + f_2$) expands the beam diameter.

In [ ]:
L1 = co.Lens("Input", [0.2, 0.0], [1, 0], 0.3, 1.5, 0.2, -0.2, 0.05) # Small Lens
L2 = co.Lens("Output", [0.8, 0.0], [1, 0], 0.6, 1.5, 0.4, -0.4, 0.05) # Large Lens
screen = co.Screen("Check", [1.2, 0.0], [-1, 0], 1.0)

rays = co.generate_parallel_beam([(550, 'green')], [0,0], [0.2,0], 0.15, 10)
co.run_simulation(rays, [L1, L2, screen], bounces=10, use_gpu=False)

fig, ax = plt.subplots(figsize=(10, 4))
co.plot_system(rays, [L1, L2, screen], ax)
plt.title("Beam Expander")
plt.show()

### Example B: Folded Spectrometer
Light enters -> Reflects off mirror -> Hits Grating -> Focuses on Detector.

In [ ]:
mirror = co.FlatMirror("Fold Mirror", [0.5, 0.0], [-1, 1], 0.4) # 45 deg angle
grating = co.ReflectiveGrating("Grating", [0.5, 0.5], [0, -1], 0.4, 600, 2)
screen = co.Screen("Detector", [0.8, 0.2], [-1, 0], 0.5)

rays = co.generate_parallel_beam([(450, 'blue'), (650, 'red')], [0.0, 0.0], [0.5, 0.0], 0.1, 5)
co.run_simulation(rays, [mirror, grating, screen], bounces=5, use_gpu=False)

fig, ax = plt.subplots(figsize=(8, 8))
co.plot_system(rays, [mirror, grating, screen], ax)
plt.title("Folded Spectrometer")
plt.show()